In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json
import pickle


# Set up your credentials
consumer_key    = 'rFXo7y4lDLzBCa6edAbyJPXO6'
consumer_secret = 'kRgSC8luGa3J5pFu4AEE37YoAeRs9HkFmgoM5cg5Nv2ieNGUaU'
access_token    = '1042056400560680961-AdlgRfGTiyfj6aE0nmOylj2EEtTbeK'
access_secret   = 'Uquvk5RrlhIT4oFsMm7VNBCRe8r4rMibXJXZhPau0culW'


class TweetsListener(StreamListener):

  def __init__(self, csocket):
      self.client_socket = csocket

  def on_data(self, data):
      try:
          s=self.client_socket  
          s.listen(5)                 # Now wait for client connection.
          c, addr = s.accept()        # Establish connection with client.

          print("Received request from: " + str(addr))
            
          msg = json.loads( data )
#           print(msg)
          user=json.loads( json.dumps(msg['user']) )
          sdata=msg['text'].replace('\n','')+' ~@ '+(user['location'] if user['location'] is not None else 'None')+' ~@ '+msg['source']
          print(sdata.encode('utf-8'))
          
          c.send(sdata.encode('utf-8'))
          c.close()
      except IncompleteRead:
        # Oh well, reconnect and keep trucking
          return true  
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True

  def on_error(self, status):
      print(status)
      return True

def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)

  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track=['sports','football','cricket','badminton','tennis','hockey','baseball','golf'])

if __name__ == "__main__":
  s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)         # Create a socket object
  host = "127.0.0.1"     # Get local machine name
  port = 5555                 # Reserve a port for your service.
  s.bind((host, port))        # Bind to the port

  print("Listening on port: %s" % str(port))

  sendData(s)